In [1]:
using DrWatson
@quickactivate "Doran_etal_2022"

using CSV, DataFrames

In [56]:
csbobs = CSV.read(datadir("exp_pro","BB669","BB669_rowmeta_old.csv"), DataFrame);
CSV.write(datadir("exp_pro","BB669","BB669_rowmeta_old.csv"), csbobs);

In [39]:
ncbiobs = CSV.read(datadir("exp_raw","BB669","ncbi_tax.csv"), DataFrame, missingstring="NA")
gtdbobs = CSV.read(datadir("exp_raw","BB669","gtdb_tax.csv"), DataFrame);

In [40]:
size(gtdbobs)

(1864, 18)

In [41]:
length(unique(gtdbobs.msk_id))

1728

In [50]:
ncbi_taxonomy = ncbiobs[:, [:seq_id, :Phylum, :Class, :Order, :Family, :Genus, :Species]] |>
    df -> rename(df, 
        :Phylum => :phylum_ncbi, 
        :Class  => :class_ncbi, 
        :Order  => :order_ncbi,
        :Family => :family_ncbi,
        :Genus  => :genus_ncbi,
        :Species  => :species_ncbi,
    ) |> unique

gtdb_taxonomy = gtdbobs[:, [:seq_id, :Phylum, :Class, :Order, :Family, :Genus, :Species]] |>
    df -> rename(df, 
        :Phylum => :phylum_gtdb, 
        :Class  => :class_gtdb, 
        :Order  => :order_gtdb,
        :Family => :family_gtdb,
        :Genus  => :genus_gtdb,
        :Species  => :species_gtdb,
    ) |> unique;

In [54]:
mrgdf = leftjoin(csbobs, ncbi_taxonomy, on=:seq_id, makeunique=true) |>
    df->leftjoin(df, gtdb_taxonomy, on=:seq_id) |>
    df->coalesce.(df, "")
mrgdf.family_ncbi[mrgdf.genus_ncbi .== "Phocaeicola"] .= "Bacteroidaceae";
mrgdf

Row,ID,first_strain_id,strain_id,seq_id,donor_id,accession,phylum,class,order,family,genus,species,donor,phylum_ncbi,class_ncbi,order_ncbi,family_ncbi,genus_ncbi,species_ncbi,phylum_gtdb,class_gtdb,order_gtdb,family_gtdb,genus_gtdb,species_gtdb
,String15,String,String15,String31,String7,String15,String15,String31,String31,String,String,String,String7,String,String,String,String,String,String,String,String,String,String,String,String
1,DFI.4.132,DFI.4.132_Phocaeicola_dorei,DFI.4.132,210119-DFI.4.132,DFI0004,JAHONO000000000,Bacteroidetes,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides,Bacteroides dorei,DFI.4,Bacteroidetes,Bacteroidia,Bacteroidales,Bacteroidaceae,Phocaeicola,Phocaeicola dorei,Bacteroidota,Bacteroidia,Bacteroidales,Bacteroidaceae,Phocaeicola,Phocaeicola dorei
2,DFI.4.155,DFI.4.155_Anaerostipes_hadrus,DFI.4.155,210119-DFI.4.155,DFI0004,JAJCRN000000000,Firmicutes,Clostridia,Clostridiales,Lachnospiraceae,Anaerostipes,Anaerostipes hadrus,DFI.4,Firmicutes,Clostridia,Eubacteriales,Lachnospiraceae,Anaerostipes,Anaerostipes hadrus,Firmicutes_A,Clostridia,Lachnospirales,Lachnospiraceae,Anaerostipes,Anaerostipes hadrus
3,DFI.4.81,DFI.4.81_Phocaeicola_vulgatus,DFI.4.81,210119-DFI.4.81,DFI0004,JAHONG000000000,Bacteroidetes,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides,Bacteroides vulgatus,DFI.4,Bacteroidetes,Bacteroidia,Bacteroidales,Bacteroidaceae,Phocaeicola,Phocaeicola vulgatus,Bacteroidota,Bacteroidia,Bacteroidales,Bacteroidaceae,Phocaeicola,Phocaeicola vulgatus
4,DFI.1.135,DFI.1.135_Bacteroides_uniformis,DFI.1.135,210120-DFI.1.135,DFI0001,JAHOOY000000000,Bacteroidetes,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides,Bacteroides uniformis,DFI.1,Bacteroidetes,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides,Bacteroides uniformis,Bacteroidota,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides,Bacteroides uniformis
5,DFI.1.156,DFI.1.156_Odoribacter_splanchnicus,DFI.1.156,210120-DFI.1.156,DFI0001,JAHOOW000000000,Bacteroidetes,Bacteroidia,Bacteroidales,Odoribacteraceae,Odoribacter,Odoribacter splanchnicus,DFI.1,Bacteroidetes,Bacteroidia,Bacteroidales,Odoribacteraceae,Odoribacter,Odoribacter splanchnicus,Bacteroidota,Bacteroidia,Bacteroidales,Marinifilaceae,Odoribacter,Odoribacter splanchnicus
6,DFI.1.157,DFI.1.157_Odoribacter_splanchnicus,DFI.1.157,210120-DFI.1.157,DFI0001,JAHOOV000000000,Bacteroidetes,Bacteroidia,Bacteroidales,Odoribacteraceae,Odoribacter,Odoribacter splanchnicus,DFI.1,Bacteroidetes,Bacteroidia,Bacteroidales,Odoribacteraceae,Odoribacter,Odoribacter splanchnicus,Bacteroidota,Bacteroidia,Bacteroidales,Marinifilaceae,Odoribacter,Odoribacter splanchnicus
7,DFI.1.247,DFI.1.247_Bacteroides_uniformis,DFI.1.247,210120-DFI.1.247,DFI0001,JAHOOT000000000,Bacteroidetes,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides,Bacteroides uniformis,DFI.1,Bacteroidetes,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides,Bacteroides uniformis,Bacteroidota,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides,Bacteroides uniformis
8,DFI.1.37,DFI.1.37_Parabacteroides_distasonis,DFI.1.37,210120-DFI.1.37,DFI0001,JAHOOS000000000,Bacteroidetes,Bacteroidia,Bacteroidales,Tannerellaceae,Parabacteroides,Parabacteroides distasonis,DFI.1,Bacteroidetes,Bacteroidia,Bacteroidales,Tannerellaceae,Parabacteroides,Parabacteroides distasonis,Bacteroidota,Bacteroidia,Bacteroidales,Tannerellaceae,Parabacteroides,Parabacteroides distasonis
9,DFI.1.54,DFI.1.54_Bacteroides_thetaiotaomicron,DFI.1.54,210120-DFI.1.54,DFI0001,JAHOOR000000000,Bacteroidetes,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides,Bacteroides thetaiotaomicron,DFI.1,Bacteroidetes,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides,Bacteroides thetaiotaomicron,Bacteroidota,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides,Bacteroides thetaiotaomicron


In [57]:
CSV.write(datadir("exp_pro","BB669","BB669_rowmeta.csv"), mrgdf);

In [58]:
CSV.write(datadir("exp_raw","BB669","BB669_rowmeta.csv"), mrgdf);